In [1]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
summoner_name = "야식은치킨이지"
match = pd.read_json("data/{name}/match.json".format(name=summoner_name))
timeline = pd.read_json("data/{name}/timeline.json".format(name=summoner_name))

ValueError: Expected object or value

In [ ]:
# match 데이터를 저장할 때 index 부분을 조정해주어야 할 듯 
for i in range(100):
    match.iloc[i,0] = (match.iloc[i,0]-1)%10+1

match.head()

# 경기 수 관련

In [ ]:
match_length = len(timeline)
match_invade = 0

# 이벤트 분석

In [ ]:
match_index = 1
match_time_length = len(timeline.iloc[match_index].info['frames'])
team_id = match.teamId[match_index]
win = match.win[match_index]

events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

In [ ]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    team_id = match.teamId[match_index]
    win = match.win[match_index]
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    for t in range(1,3):  # 1 ~ 2 분 사이
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1
                # display(pd.json_normalize(e))

    if is_invade[match_index]:
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        if team_id == 100:
            if win: invade_win_team[match_index] = 100
            else: invade_win_team[match_index] = 200
        else:
            if win: invade_win_team[match_index] = 200
            else: invade_win_team[match_index] = 100    
        print("--------------------------------------------------")
        print("invade detected: %f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% "blue" if invade_win_team[match_index] == 100 else "red")

In [ ]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


In [ ]:
match_invade